In [1]:
from utils import get_spark_session
import time
from data_generator import gen_bulk_data

spark = get_spark_session("iceberg_nessie_branching")
df_test = gen_bulk_data(spark, 10**4)

:: loading settings :: url = jar:file:/opt/bitnami/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.12 added as a dependency
software.amazon.awssdk#bundle added as a dependency
software.amazon.awssdk#url-connection-client added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9c2759e7-67bd-4006-9621-30205071ac19;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.6.1 in central
	found org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.12;0.95.0 in central
	found software.amazon.awssdk#bundle;2.17.178 in central
	found software.amazon.eventstream#eventstream;1.0.1 in central
	found software.amazon.awssdk#url-connection-client;2.17.178 in central
	found software.amazon.awssdk#utils;2.17.178 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found softwa

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- salary: double (nullable = true)
 |-- last_purchase: timestamp (nullable = true)
 |-- last_purchase_date: string (nullable = true)



### Branching steps

1. Create a Spark Dataframe;
2. Load the data into a TEMPORARY VIEW;
3. Create a table using CTAS statement and the view created above;
4. 


In [2]:

TABLE_NAME = "nessie.dev.branching"
df_test.createOrReplaceTempView("df_test")

spark.sql("SELECT * FROM df_test").show()

spark.sql("USE REFERENCE main IN nessie").show()

spark.sql(f"CREATE OR REPLACE TABLE {TABLE_NAME} USING iceberg AS SELECT * FROM df_test").show()
print(f"Num lines after creation: {spark.table('nessie.dev.branching').count()}")

spark.sql("CREATE BRANCH IF NOT EXISTS feature_other IN nessie").show()
spark.sql("USE REFERENCE feature_other IN nessie").show()

spark.sql(f"DELETE FROM {TABLE_NAME} WHERE age > 50")
print(f"Num lines after deletion in feature branch: {spark.table(TABLE_NAME).count()}")

spark.sql("USE REFERENCE main IN nessie").show()
print(f"Num lines after deletion in main branch: {spark.table(TABLE_NAME).count()}")

spark.sql("MERGE BRANCH feature_other INTO main IN nessie")
time.sleep(30)

print(f"Num lines after merge of feature_other into main branch: {spark.table(TABLE_NAME).count()}")



+-------+--------------------+---+-------+-------------------+------------------+
|     id|                name|age| salary|      last_purchase|last_purchase_date|
+-------+--------------------+---+-------+-------------------+------------------+
|3343488|       Maysa da Cruz| 58|1004.11|1972-08-08 00:19:06|        1972-08-08|
|7937829|       Sofia da Rosa| 37| 996.98|1971-05-13 02:51:54|        1971-05-13|
|7253113|Carlos Eduardo Ramos| 55|1005.93|1976-08-07 03:14:02|        1976-08-07|
|7993723| Davi Miguel Costela| 44|1003.76|1973-02-09 14:45:42|        1973-02-09|
|6938021|        Ayla Pimenta| 16| 993.98|1978-06-29 08:08:44|        1978-06-29|
|4513143|Srta. Evelyn Nasc...| 78| 997.23|1990-11-09 14:44:47|        1990-11-09|
| 721575|      Miguel Sampaio| 57| 991.53|1995-01-29 07:23:03|        1995-01-29|
|5126821| José Pedro Monteiro| 61|  993.2|1995-06-11 12:22:44|        1995-06-11|
|7965086|    Sabrina Nogueira| 28|1008.09|1988-10-07 22:43:38|        1988-10-07|
|8496275|       

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".                
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


++
||
++
++

Num lines after creation: 10000
+-------+-------------+--------------------+
|refType|         name|                hash|
+-------+-------------+--------------------+
| Branch|feature_other|85aaa5c12490b657d...|
+-------+-------------+--------------------+

+-------+-------------+--------------------+
|refType|         name|                hash|
+-------+-------------+--------------------+
| Branch|feature_other|85aaa5c12490b657d...|
+-------+-------------+--------------------+



Num lines after deletion in feature branch: 5093
+-------+----+--------------------+
|refType|name|                hash|
+-------+----+--------------------+
| Branch|main|85aaa5c12490b657d...|
+-------+----+--------------------+

Num lines after deletion in main branch: 10000
Num lines after merge of feature_other into main branch: 5093


In [3]:
print(f"Num lines after merge of feature_other into main branch: {spark.table(TABLE_NAME).count()}")

spark.sql("DROP BRANCH IF EXISTS feature_other IN nessie").show()
spark.sql("DROP TABLE nessie.dev.branching").show()

Num lines after merge of feature_other into main branch: 5093
+------+
|status|
+------+
|    OK|
+------+

++
||
++
++



In [4]:
spark.stop()